# Toronto Neighbourhoods Project
## This project will be used for the Coursera final project

In [141]:
import pandas as pd
import numpy as np
import geocoder as ge
import requests
import json
from sklearn.cluster import KMeans
import folium 
import matplotlib.cm as cm
import matplotlib.colors as colors
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim 


Creating and fixing the data

In [142]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url)
df= pd.DataFrame(df[0])
df=df[df['Borough']!='Not assigned'] #Removes all Not assigned Values
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [143]:
def get_geocode(postal_code):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [144]:
geo=pd.read_csv('http://cocl.us/Geospatial_data')
geo.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [145]:
df1=df.groupby('Postal Code')['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
df1=df1.reset_index(drop=False)
df1.rename(columns={'Neighbourhood':'Neighbourhood_joined'},inplace=True)
df2 = pd.merge(df, df1, on='Postal Code')
df2.drop(['Neighbourhood'],axis=1,inplace=True)
df2.drop_duplicates(inplace=True)
df2.rename(columns={'Neighbourhood_joined':'Neighbourhood'},inplace=True)
df2.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [146]:
def get_latlng(neigh):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = ge.arcgis('{}, Toronto, Ontario, Canada'.format(neigh))
        lat_lng_coords = g.latlng
    return lat_lng_coords
coords = [ get_latlng(neigh) for neigh in df2['Neighbourhood']]
df3 = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df2['Latitude'] = df3['Latitude']
df2['Longitude'] = df3['Longitude']
print(df2.shape)
df2.head(456)

(103, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.686588,-79.409996
1,M4A,North York,Victoria Village,43.731540,-79.314280
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.659743,-79.361561
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.714925,-79.449739
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.666622,-79.393264
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.651321,-79.523468
99,M4Y,Downtown Toronto,Church and Wellesley,43.665700,-79.380930
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.648690,-79.385440
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.650520,-79.484960


In [147]:
CLIENT_ID = 'IJ5XCHFSSXUAUIYUYYSOUCVSGLZK241BNGEEGT252CIY14N5' 
CLIENT_SECRET = 'CRXPEJBKAAKI1YLZGSRO0QWOC22AVGJ3SG1OGENV2PBJGUNP' 
VERSION = '20180605'
LIMIT = 100 
neighborhood_latitude = df2.loc[0, 'Latitude']
neighborhood_longitude = df2.loc[0, 'Longitude']
neighborhood_name = df2.loc[0, 'Neighbourhood']
radius = 500 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude, neighborhood_longitude, radius, LIMIT)

In [148]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fb4735d526d023dff8a73cf'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Forest Hill South',
  'headerFullLocation': 'Forest Hill South, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 22,
  'suggestedBounds': {'ne': {'lat': 43.691088494171304,
    'lng': -79.40378485696377},
   'sw': {'lat': 43.6820884851713, 'lng': -79.41620754625737}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e81e6c1469032a28cc3fc95',
       'name': 'Aroma Espresso Bar',
       'location': {'address': '383 Spadina Road',
        'crossStreet': 'Lonsdale',
        'lat': 43.68816960127832,
        'lng': -79.41259869674182,
        'labele

In [149]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head(54)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  del sys.path[0]


,name,categories,lat,lng
0,Aroma Espresso Bar,Café,43.688170,-79.412599
1,Sir Winston Churchill Park,Park,43.683732,-79.409881
2,Mashu Mashu Mediterranean Grill,Middle Eastern Restaurant,43.688297,-79.412563
3,What A Bagel,Bagel Shop,43.688079,-79.414544
4,Starbucks,Coffee Shop,43.688970,-79.413097
5,Track Fitness,Gym,43.688744,-79.412607
6,Hero Certified Burgers,Burger Joint,43.688631,-79.412636
7,Pizza Banfi,Italian Restaurant,43.688064,-79.414547
8,TD Canada Trust,Bank,43.689333,-79.412751
9,RBC Royal Bank,Bank,43.688705,-79.413090


In [150]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,  CLIENT_SECRET, VERSION, lat,  lng, radius, LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'],  v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude',  'Venue Category']   
    return(nearby_venues)

In [151]:
toronto_venues = getNearbyVenues(names=df2['Neighbourhood'],latitudes=df2['Latitude'],longitudes=df2['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [152]:
print(toronto_venues.shape)
toronto_venues.head()

(2614, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.686588,-79.409996,Aroma Espresso Bar,43.688170,-79.412599,Café
1,Parkwoods,43.686588,-79.409996,Sir Winston Churchill Park,43.683732,-79.409881,Park
2,Parkwoods,43.686588,-79.409996,Mashu Mashu Mediterranean Grill,43.688297,-79.412563,Middle Eastern Restaurant
3,Parkwoods,43.686588,-79.409996,What A Bagel,43.688079,-79.414544,Bagel Shop
4,Parkwoods,43.686588,-79.409996,Starbucks,43.688970,-79.413097,Coffee Shop


In [153]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,17,17,17,17,17,17
"Alderwood, Long Branch",15,15,15,15,15,15
"Bathurst Manor, Wilson Heights, Downsview North",13,13,13,13,13,13
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",17,17,17,17,17,17
...,...,...,...,...,...,...
"Willowdale, Willowdale West",2,2,2,2,2,2
Woburn,10,10,10,10,10,10
Woodbine Heights,18,18,18,18,18,18


In [154]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head(5454)

,Yoga Studio,Afghan Restaurant,African Restaurant,Airport Service,American Restaurant,Animal Shelter,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,...,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2609,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2610,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2611,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2612,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [155]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,African Restaurant,Airport Service,American Restaurant,Animal Shelter,Antique Shop,Art Gallery,Art Museum,...,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.058824,0.0,0.0,0.000000,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.066667,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
3,Bayview Village,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,"Willowdale, Willowdale West",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
94,Woburn,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
95,Woodbine Heights,0.0,0.0,0.055556,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
96,York Mills West,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0


In [156]:
num_top_venues = 5
for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                 venue  freq
0   Chinese Restaurant  0.24
1     Asian Restaurant  0.18
2          Coffee Shop  0.06
3               Bakery  0.06
4  Rental Car Location  0.06


----Alderwood, Long Branch----
           venue  freq
0           Bank  0.13
1  Grocery Store  0.13
2    Wings Joint  0.07
3     Beer Store  0.07
4       Pharmacy  0.07


----Bathurst Manor, Wilson Heights, Downsview North----
                 venue  freq
0             Pharmacy  0.15
1          Coffee Shop  0.15
2        Deli / Bodega  0.08
3       Sandwich Place  0.08
4  Filipino Restaurant  0.08


----Bayview Village----
                        venue  freq
0  Construction & Landscaping  0.25
1                 Flower Shop  0.25
2          Golf Driving Range  0.25
3                       Trail  0.25
4                Music School  0.00


----Bedford Park, Lawrence Manor East----
               venue  freq
0   Department Store  0.12
1         Bagel Shop  0.12
2         Kids Store  0.06
3  Electron

                  venue  freq
0  Fast Food Restaurant  0.21
1              Pharmacy  0.14
2           Pizza Place  0.14
3            Restaurant  0.07
4           Supermarket  0.07


----Milliken, Agincourt North, Steeles East, L'Amoreaux East----
                     venue  freq
0              Pizza Place  0.50
1                     Bank  0.25
2     Fast Food Restaurant  0.25
3                   Museum  0.00
4  New American Restaurant  0.00


----Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West----
                   venue  freq
0             Playground   0.2
1                    Bar   0.2
2                 Bakery   0.2
3  Performing Arts Venue   0.2
4                   Café   0.2


----Moore Park, Summerhill East----
               venue  freq
0               Park  0.25
1  Convenience Store  0.06
2  Indian Restaurant  0.06
3    Thai Restaurant  0.06
4   Sushi Restaurant  0.06


----New Toronto, Mimico South, Humber Bay Shores----
         

                        venue  freq
0                  Playground   0.5
1                       Hotel   0.5
2   Middle Eastern Restaurant   0.0
3           Mobile Phone Shop   0.0
4  Modern European Restaurant   0.0


----Willowdale, Willowdale West----
                     venue  freq
0                     Park   0.5
1        Mobile Phone Shop   0.5
2            Moving Target   0.0
3  New American Restaurant   0.0
4               Nail Salon   0.0


----Woburn----
                        venue  freq
0           Indian Restaurant   0.2
1     South Indian Restaurant   0.2
2  Construction & Landscaping   0.1
3                      Bakery   0.1
4           French Restaurant   0.1


----Woodbine Heights----
         venue  freq
0  Coffee Shop  0.17
1         Café  0.11
2    Gastropub  0.11
3      Library  0.06
4         Bank  0.06


----York Mills West----
              venue  freq
0       Coffee Shop  0.17
1              Park  0.11
2      Intersection  0.06
3  Business Service  0.06
4     

In [157]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)   
    return row_categories_sorted.index.values[0:num_top_venues]
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']
for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Asian Restaurant,Food Court,Intersection,Bakery,Cantonese Restaurant,Coffee Shop,Korean Restaurant,Rental Car Location,Hong Kong Restaurant
1,"Alderwood, Long Branch",Bank,Grocery Store,Beer Store,Wings Joint,Pharmacy,Italian Restaurant,Restaurant,Greek Restaurant,Pizza Place,Café
2,"Bathurst Manor, Wilson Heights, Downsview North",Pharmacy,Coffee Shop,Filipino Restaurant,Café,Restaurant,Beer Store,Sandwich Place,Bagel Shop,Asian Restaurant,Deli / Bodega
3,Bayview Village,Trail,Construction & Landscaping,Flower Shop,Golf Driving Range,Women's Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
4,"Bedford Park, Lawrence Manor East",Department Store,Bagel Shop,Breakfast Spot,Supermarket,Sandwich Place,Bank,Kids Store,Coffee Shop,Electronics Store,Doctor's Office


In [158]:
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10] 
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = df2
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
toronto_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.686588,-79.409996,4.0,Park,Bank,Café,Middle Eastern Restaurant,Sushi Restaurant,Bookstore,Ice Cream Shop,Coffee Shop,Bagel Shop,Japanese Restaurant
1,M4A,North York,Victoria Village,43.731540,-79.314280,2.0,Park,Women's Store,Escape Room,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.659743,-79.361561,4.0,Coffee Shop,Thai Restaurant,Pharmacy,Performing Arts Venue,Pet Store,Pool,Pub,Restaurant,Electronics Store,Food Truck
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.714925,-79.449739,4.0,Fast Food Restaurant,Restaurant,Coffee Shop,Pet Store,Department Store,Mobile Phone Shop,Flower Shop,Moving Target,Bookstore,Sandwich Place
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.666622,-79.393264,4.0,Coffee Shop,Café,Clothing Store,Boutique,Restaurant,Italian Restaurant,Pizza Place,Park,Spa,Cosmetics Shop


In [161]:
toronto_merged.dropna(inplace = True)

In [162]:
# create map
toronto_merged.dropna()
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'],toronto_merged['Neighbourhood'],toronto_merged['Cluster Labels'].astype(int)):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters